In [1]:
import sys

sys.path.append("../")

In [2]:
import torch

import simulation
from utils import set_random_seed_all
from train_utils import (
    create_intervention_dataset,
    subset_interventions,
)

from models import SDCD

In [3]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [4]:
n, d = 50, 500
n_edges = d * 5
frac_interventions = 1.0
knockdown_eff = 1.0

B_true = simulation.simulate_dag(d, n_edges, "ER")
X_full_df, param_dict = simulation.generate_full_interventional_set(
    B_true, n, "mlp", knockdown_eff=knockdown_eff, size_observational=n * (d + 1)
)
n_interventions = int(frac_interventions * d)
X_df = subset_interventions(
    X_full_df, n_interventions, maintain_dataset_size=True
)

In [5]:
X_df

,0,1,2,3,4,5,6,7,8,9,...,491,492,493,494,495,496,497,498,499,perturbation_label
25050,-0.000000,14.618263,2.526590,9.105727,-2.740230,-27.656484,0.378563,-7.967108,-10.010663,-3.580402,...,-8.104380,-10.294197,0.132198,4.839391,-7.252841,0.357367,-5.733964,-1.004192,6.403715,0
25051,-0.000000,10.580520,2.883131,10.867246,-3.782385,-9.632183,-0.787190,-13.177756,-9.186022,-5.010482,...,-7.471177,-7.503429,-9.114888,-2.996159,-6.145024,-2.879723,-5.620252,-3.020078,9.394594,0
25052,-0.000000,12.534614,0.271269,8.642501,-6.132475,-16.589926,1.054897,-7.349479,-1.298002,-5.182514,...,-4.104870,-10.700136,0.659528,-0.049770,-5.479820,-4.372787,-11.567073,4.082894,9.135345,0
25053,-0.000000,13.226782,7.393634,5.039977,-2.214171,-9.730682,1.402239,-12.534208,-7.742577,-4.388275,...,-8.028435,3.947090,-13.416799,1.197753,-9.831138,-2.241728,-12.734417,5.722317,5.936777,0
25054,-0.000000,14.647099,-1.746368,12.524058,-0.796306,-17.714186,1.735530,-10.605568,-5.204759,-3.779962,...,-5.035220,-2.154080,-8.610936,4.363183,-2.472915,2.962205,-9.321836,0.976773,8.346799,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23172,-3.487573,18.350298,1.947800,10.971582,-3.807026,-15.701888,1.263148,-10.557827,-2.335702,-6.303080,...,-8.308770,-10.500962,-7.467567,1.069525,-7.815384,2.298159,-7.776415,6.719611,5.257911,obs
19913,-4.455120,7.406244,-1.421838,5.157877,1.664816,-10.333174,-1.136816,-12.730028,-9.568093,-4.794345,...,-6.790865,1.754474,3.640750,6.305562,-5.023647,-1.325560,-10.388608,-1.891084,6.310453,obs
23600,-2.417609,11.795030,3.612644,8.946375,-9.401715,-8.560443,-1.250613,-13.556851,-17.632403,-4.748931,...,-3.812272,-9.285847,-5.980075,1.701713,-3.538776,-7.354235,-10.295281,1.155350,6.795298,obs
1195,-4.097208,8.195340,-2.424775,3.654040,-4.977332,-5.777910,-0.278543,-8.124676,-3.661279,-2.791880,...,-6.912916,-0.466438,-6.297675,0.322356,-6.475673,-0.260729,-14.921145,1.567944,8.250785,obs


In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [7]:
dataset = create_intervention_dataset(X_df, regime_format=False)

In [8]:
model = SDCD()
model.train(
    dataset,
    B_true=B_true,
    device=device,
    stage1_kwargs={"n_epochs": 100},
    stage2_kwargs={"n_epochs": 100}
)

/home/justin/miniconda3/envs/cp/lib/python3.9/site-packages/torch/optim/optimizer.py:243: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  if not is_compiling() and torch.has_cuda and torch.cuda.is_available():


Epoch 0: loss=30479.30, score=2528, shd=2528.0, gamma=0.00
Recall of mask: 0.934
Fraction of possible edges in mask: 0.01802
Epoch 0: loss=21013.35, score=2364, shd=2364.0, gamma=0.00
Finished training in 58.55799078941345 seconds.


In [9]:
model = SDCD()
model.train(
    dataset,
    B_true=B_true,
    device="cpu",
    stage1_kwargs={"n_epochs": 100},
    stage2_kwargs={"n_epochs": 100}
)

Epoch 0: loss=30488.05, score=2538, shd=2538.0, gamma=0.00
Recall of mask: 0.9364
Fraction of possible edges in mask: 0.017876
Epoch 0: loss=20833.06, score=2302, shd=2302.0, gamma=0.00
Finished training in 288.62951254844666 seconds.
